# Chapter 12 - Custom Models and Training with TensorFlow

## Part I: Tensors and Operations

## Setup

In [1]:
import sys
assert sys.version_info >= (3, 5)

In [2]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [3]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= '2.0'

In [4]:
import numpy as np
import os

In [5]:
# To make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [7]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

In [8]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Tensors and Operations

### Tensors:

In [9]:
tf.constant([[1., 2., 3.],
             [4., 5., 6.]])

<tf.Tensor: id=0, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [10]:
tf.constant(42)

<tf.Tensor: id=2, shape=(), dtype=int32, numpy=42>

In [11]:
t = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])

In [12]:
t.shape

TensorShape([2, 3])

In [13]:
t.dtype

tf.float32

### Indexing:

In [14]:
t[:, 1:]

<tf.Tensor: id=10, shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [16]:
t[..., 1, tf.newaxis]

<tf.Tensor: id=20, shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

### Operations:

In [17]:
t + 10

<tf.Tensor: id=23, shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [18]:
tf.square(t)

<tf.Tensor: id=25, shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [19]:
t @ tf.transpose(t)

<tf.Tensor: id=29, shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### Using `keras.backend`:

In [20]:
K = keras.backend

In [21]:
K.square(K.transpose(t)) + 10

<tf.Tensor: id=35, shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### From/To NumPy:

In [22]:
a = np.array([2., 4., 5.])

In [23]:
tf.constant(a)

<tf.Tensor: id=37, shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [24]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [25]:
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [26]:
tf.square(a)

<tf.Tensor: id=43, shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [27]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Conflicting Types:

In [29]:
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute Add as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:Add] name: add/


In [30]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute Add as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:Add] name: add/


In [32]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: id=55, shape=(), dtype=float32, numpy=42.0>

### Strings:

In [33]:
tf.constant(b'hello world')

<tf.Tensor: id=57, shape=(), dtype=string, numpy=b'hello world'>

In [34]:
tf.constant('cafe')

<tf.Tensor: id=59, shape=(), dtype=string, numpy=b'cafe'>

In [35]:
u = tf.constant([ord(c) for c in 'cafe'])
u

<tf.Tensor: id=61, shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 101], dtype=int32)>

In [36]:
b = tf.strings.unicode_encode(u, 'UTF-8')
tf.strings.length(b, unit='UTF8_CHAR')

<tf.Tensor: id=73, shape=(), dtype=int32, numpy=4>

In [37]:
tf.strings.unicode_decode(b, 'UTF-8')

<tf.Tensor: id=78, shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 101], dtype=int32)>

### String Arrays:

In [38]:
p = tf.constant(['Cafe', 'Coffee', 'caffe', '咖啡'])

In [39]:
tf.strings.length(p, unit='UTF8_CHAR')

<tf.Tensor: id=81, shape=(4,), dtype=int32, numpy=array([4, 6, 5, 2], dtype=int32)>

In [40]:
r = tf.strings.unicode_decode(p, 'UTF8')
r

tf.RaggedTensor(values=tf.Tensor(
[   67    97   102   101    67   111   102   102   101   101    99    97
   102   102   101 21654 21857], shape=(17,), dtype=int32), row_splits=tf.Tensor([ 0  4 10 15 17], shape=(5,), dtype=int64))

### Ragged Tensors:

In [41]:
print(r[1])

tf.Tensor([ 67 111 102 102 101 101], shape=(6,), dtype=int32)


In [42]:
print(r[1:3])

<tf.RaggedTensor [[67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 101]]>


In [44]:
r2 = tf.ragged.constant([[68, 69, 70],
                         [71],
                         [],
                         [72, 73]])

In [45]:
print(tf.concat([r, r2], axis=0))

<tf.RaggedTensor [[67, 97, 102, 101], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 101], [21654, 21857], [68, 69, 70], [71], [], [72, 73]]>


In [46]:
r3 = tf.ragged.constant([[68, 69, 70], [71], [], [72, 73]])

In [47]:
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[67, 97, 102, 101, 68, 69, 70], [67, 111, 102, 102, 101, 101, 71], [99, 97, 102, 102, 101], [21654, 21857, 72, 73]]>


In [48]:
tf.strings.unicode_encode(r3, 'UTF-8')

<tf.Tensor: id=242, shape=(4,), dtype=string, numpy=array([b'DEF', b'G', b'', b'HI'], dtype=object)>

In [49]:
r.to_tensor()

<tf.Tensor: id=310, shape=(4, 6), dtype=int32, numpy=
array([[   67,    97,   102,   101,     0,     0],
       [   67,   111,   102,   102,   101,   101],
       [   99,    97,   102,   102,   101,     0],
       [21654, 21857,     0,     0,     0,     0]], dtype=int32)>

### Sparse Tensors:

In [50]:
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]],
                    values=[1., 2., 3.],
                    dense_shape=[3, 4])

In [51]:
s

In [52]:
print(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [53]:
tf.sparse.to_dense(s)

<tf.Tensor: id=319, shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

In [54]:
s2 = s * 2.0

In [55]:
try:
    s3 = s + 1
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'SparseTensor' and 'int'


In [56]:
s4 = tf.constant([[10., 20.], [30., 40.], [50., 60.], [70., 80.]])
tf.sparse.sparse_dense_matmul(s, s4)

<tf.Tensor: id=324, shape=(3, 2), dtype=float32, numpy=
array([[ 30.,  40.],
       [ 20.,  40.],
       [210., 240.]], dtype=float32)>

In [57]:
s5 = tf.SparseTensor(indices=[[0, 2], [0, 1]],
                     values=[1., 2.],
                     dense_shape=[3, 4])

In [58]:
print(s5)

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]], shape=(2, 2), dtype=int64), values=tf.Tensor([1. 2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [60]:
try:
    tf.sparse.to_dense(s5)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

indices[1] = [0,1] is out of order [Op:SparseToDense]


In [61]:
s6 = tf.sparse.reorder(s5)
tf.sparse.to_dense(s6)

<tf.Tensor: id=340, shape=(3, 4), dtype=float32, numpy=
array([[0., 2., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Sets:

In [62]:
set1 = tf.constant([[2, 3, 5, 7],
                    [7, 9, 0, 0]])
set2 = tf.constant([[4, 5, 6],
                    [9, 10, 0]])

In [63]:
tf.sparse.to_dense(tf.sets.union(set1, set2))

<tf.Tensor: id=348, shape=(2, 6), dtype=int32, numpy=
array([[ 2,  3,  4,  5,  6,  7],
       [ 0,  7,  9, 10,  0,  0]], dtype=int32)>

In [64]:
tf.sparse.to_dense(tf.sets.intersection(set1, set2))

<tf.Tensor: id=354, shape=(2, 2), dtype=int32, numpy=
array([[5, 0],
       [0, 9]], dtype=int32)>

### Variables:

In [65]:
v = tf.Variable([[1., 2., 3.],
                 [4., 5., 6.]])

In [66]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [67]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [68]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [69]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


In [70]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

In [71]:
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.],
                                        [4., 5., 6.]],
                                indices=[1, 0])

In [72]:
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

### Tensor Arrays:

In [73]:
array = tf.TensorArray(dtype=tf.float32, size=3)

In [75]:
array = array.write(0, tf.constant([1., 2.]))
print(array)

In [76]:
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))

In [77]:
array.read(1)

<tf.Tensor: id=406, shape=(2,), dtype=float32, numpy=array([ 3., 10.], dtype=float32)>

In [78]:
array.stack()

<tf.Tensor: id=412, shape=(3, 2), dtype=float32, numpy=
array([[1., 2.],
       [0., 0.],
       [5., 7.]], dtype=float32)>

In [79]:
mean, variance = tf.nn.moments(array.stack(), axes=0)

In [80]:
mean

<tf.Tensor: id=421, shape=(2,), dtype=float32, numpy=array([2., 3.], dtype=float32)>

In [81]:
variance

<tf.Tensor: id=422, shape=(2,), dtype=float32, numpy=array([4.6666665, 8.666667 ], dtype=float32)>